# fcecodec-example for Google Colab

<a href="https://colab.research.google.com/github/bfut/notebooks/blob/main/fcecodec/fcecodec-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Basic example.

https://github.com/bfut/fcecodec</br>
https://github.com/bfut/notebooks -> ```fcecodec/fcecodec-example.ipynb```

In [ ]:
!ls -lg
!rm -rf fcecodec

Necessary for Jupyter Notebooks only, to capture output of library functions:

In [ ]:
# https://github.com/minrk/wurlitzer
!python -m pip install wurlitzer
%load_ext wurlitzer

In [ ]:
!python -m pip install --upgrade pip wheel setuptools pybind11
!git clone https://github.com/bfut/fcecodec
!python -m pip install fcecodec/python
import fcecodec

In [ ]:
filepath_fce_input = "fcecodec/tests/fce/Snowman_car.fce"
filepath_fce_output = "fcecodec/tests/fce/car_output.fce"
filepath_fce_input2 = "fcecodec/tests/fce/Snowman_car.fce"

with open(filepath_fce_input, "rb") as f:
    fce_buf = f.read()

In [ ]:
fcecodec.PrintFceInfo(fce_buf)

Load a file.

In [ ]:
mesh = fcecodec.Mesh()
mesh.IoDecode(fce_buf)

print(mesh.MNumParts)
print(mesh.MNumTriags)
print(mesh.MNumVerts)
mesh.PrintInfo()

assert mesh.MValid() == 1

Operate on data.

In [ ]:
# Merge parts 0, 3 to new part
new_pid = mesh.OpMergeParts(0, 3)
assert new_pid != -1

In [ ]:
# Copy part 
new_pid = mesh.OpCopyPart(1)
assert new_pid != -1

What has changed?

In [ ]:
print(mesh.MNumParts)
print(mesh.MNumTriags)
print(mesh.MNumVerts)
mesh.PrintInfo()

assert mesh.MValid() == 1

Another operation.

In [ ]:
# Insert/copy part 1 from mesh_src to mesh
with open(filepath_fce_input2, "rb") as f:
    fce_buf2 = f.read()
mesh_src = fcecodec.Mesh()
mesh_src.IoDecode(fce_buf2)
new_pid = mesh.OpInsertPart(mesh_src, 1)
assert new_pid != -1

Write result to disk.

In [ ]:
# Encode to FCE4M
out_buf = mesh.IoEncode_Fce4M()
with open(filepath_fce_output, "wb") as f:
    f.write(out_buf)

!cd fcecodec/tests/fce; ls -lg

In [ ]:
with open(filepath_fce_output, "rb") as f:
    fce_buf2 = f.read()
fcecodec.PrintFceInfo(fce_buf2)